In [1]:
import torch
import torchvision

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import transforms as T
import category_encoders as ce
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer

import seaborn as sns
from sklearn.compose import make_column_transformer
from IPython.display import clear_output
from sklearn.pipeline import Pipeline
from sklearn.impute import  KNNImputer,SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from keras.layers import Dense, Dropout, BatchNormalization
import numpy as np
from sklearn.preprocessing import StandardScaler
import torch.nn.functional as F
from tqdm.auto import tqdm
from keras.models import Sequential
import pandas as pd
from torch import nn
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.model_selection import train_test_split

C:\Users\Stepan\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [3]:
train=pd.read_csv('train.csv',sep=';')
test=pd.read_csv('test.csv',sep=';')

C:\Users\Stepan\AppData\Local\Temp\ipykernel_11964\3425513164.py:1: DtypeWarning: Columns (1,11,12,29,33,34,35,36,37,38,39,40,75,107,115,119,153) have mixed types. Specify dtype option on import or set low_memory=False.
  train=pd.read_csv('train.csv',sep=';')


In [4]:
train=train.dropna(how='all')
train=train.drop(columns=['f7','f10','f24','f161','f162','f163','id','f3','f4','f18','f21','f184','f185','f186','f5','f6','f8','f9'])
test=test.dropna(how='all')
test=test.drop(columns=['f7','f10','f24','f161','f162','f163','id','f3','f4','f18','f21','f184','f185','f186','f5','f6','f8','f9'])

In [5]:
train['f75'] = train['f75'].str.replace('9514', '').str.replace('FDKEFFLd', '')
train['f75'] = pd.to_numeric(train['f75'])
test['f75'] = test['f75'].str.replace('9514', '').str.replace('FDKEFFLd', '')
test['f75'] = pd.to_numeric(test['f75'])

In [6]:
CT=ColumnTransformer([
    ('1',OneHotEncoder(),['f11','f153','f29','f33','f34','f35','f36','f37','f38','f39','f40','f107','f115','f119']),
    ('2',ce.TargetEncoder(),['f12','f1'])
],remainder='passthrough')
train_new=CT.fit_transform(train.drop('target',axis=1),train.target)
train_new=pd.DataFrame(train_new,columns= CT.get_feature_names_out())
test_new=CT.transform(test)
test_new=pd.DataFrame(test_new,columns=CT.get_feature_names_out())

In [7]:
imp=KNNImputer()
train_new=imp.fit_transform(train_new)
test_new=imp.transform(test_new)

In [8]:
train_new=pd.DataFrame(train_new,columns= CT.get_feature_names_out())

In [9]:
numeric_columns = train_new.columns

In [10]:
def outlier_iqr(df, column, multiplier=1.5):
    q1 = df[column].quantile(0.05)
    q3 = df[column].quantile(0.95)
    iqr = q3 - q1
    lower_bound = q1 - multiplier * iqr
    upper_bound = q3 + multiplier * iqr
    outliers = ~df[column].between(lower_bound, upper_bound)
    return outliers, lower_bound, upper_bound

def anomalies_detection(df, columns=None, function=outlier_iqr, multiplier=1.5):
    if columns:
        columns_check = columns
    else:
        columns_check = df.columns
    anomalies_columns_table = {}
    anomalies = pd.Series(False, index=df.index, name='anomalies')
    for column in columns_check:
        outliers, lower_bound, upper_bound = function(df, column, multiplier=multiplier)
        anomalies[outliers[outliers].index] = True
        anomalies_columns_table[column] = [upper_bound, lower_bound, sum(outliers), 100 * np.mean(outliers)]
    anomalies_columns_table = pd.DataFrame(anomalies_columns_table).T
    anomalies_columns_table.columns = ['upper_bound', 'lower_bound', 'anomalies_count', 'anomalies_pct']
    return anomalies_columns_table, anomalies
def anomalies_report(anomalies):
    return print("Аномалий: {}\nПроцент аномалий:   {:.2f}%".format(sum(anomalies), 100*np.mean(anomalies))) 

std_data = train_new[numeric_columns].copy()
anomalies_columns_table, anomalies = anomalies_detection(std_data, multiplier = 3)    
std_data['is_anomalies'] = anomalies


display(anomalies_columns_table)
anomalies_report(anomalies)
display(std_data)

,upper_bound,lower_bound,anomalies_count,anomalies_pct
1__f11_Investment,4.0,-3.0,0.0,0.0
1__f11_OwnerOccupier,4.0,-3.0,0.0,0.0
1__f153_excellent,4.0,-3.0,0.0,0.0
1__f153_good,4.0,-3.0,0.0,0.0
1__f153_no data,4.0,-3.0,0.0,0.0
...,...,...,...,...
remainder__f287,595.0,-441.0,0.0,0.0
remainder__f288,8.0,-6.0,0.0,0.0
remainder__f289,240.0,-180.0,0.0,0.0
remainder__f290,602.0,-448.0,0.0,0.0


Аномалий: 4546
Процент аномалий:   22.73%


,1__f11_Investment,1__f11_OwnerOccupier,1__f153_excellent,1__f153_good,1__f153_no data,1__f153_poor,1__f153_satisfactory,1__f29_no,1__f29_yes,1__f33_no,...,remainder__f283,remainder__f284,remainder__f285,remainder__f286,remainder__f287,remainder__f288,remainder__f289,remainder__f290,remainder__f291,is_anomalies
0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,6.0,2.0,0.0,4.0,8.0,0.0,2.0,21.0,5.0,False
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,290.0,120.0,24.0,92.0,165.0,1.0,75.0,167.0,13.0,True
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,15.0,4.0,0.0,13.0,28.0,1.0,2.0,82.0,9.0,False
3,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,4.0,2.0,0.0,3.0,15.0,1.0,0.0,24.0,4.0,False
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,18.0,1.0,0.0,5.0,14.0,0.0,4.0,44.0,5.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19994,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,7.0,2.0,0.0,6.0,23.0,0.0,7.0,62.0,13.0,False
19995,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,...,5.0,2.0,0.0,6.0,12.0,0.0,2.0,44.0,5.0,False
19996,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,...,3.0,1.0,0.0,5.0,12.0,1.0,0.0,27.0,3.0,False
19997,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,32.0,12.0,3.0,10.0,25.0,1.0,8.0,83.0,2.0,True


In [11]:
std_data_cleaned = std_data[anomalies]
train_new=train_new.drop(std_data_cleaned.index)
train_new=train_new.reset_index(drop=True)

In [12]:
scaler=MinMaxScaler()
train_new=scaler.fit_transform(train_new)
test_new=scaler.transform(test_new)

C:\Users\Stepan\anaconda3\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [13]:
train_new=pd.DataFrame(train_new,columns= CT.get_feature_names_out())
train_new=pd.concat([train_new,train.target.drop(std_data_cleaned.index).reset_index(drop=True)],axis=1)


In [14]:
test_new=pd.DataFrame(test_new,columns=CT.get_feature_names_out())

AUTOENCODER

In [15]:
import torch

def rmse_loss(y_pred, y_true):
    mse = torch.mean((y_pred - y_true)**2)  # Среднеквадратичная ошибка (MSE)
    rmse = torch.sqrt(mse)  # Корень из среднеквадратичной ошибки (RMSE)
    return rmse

In [16]:
def mape(y_pred,y_val):
    absolute_percentage_errors = torch.abs((y_pred - y_val) / y_val) * 100
    mean_absolute_percentage_error = torch.mean(absolute_percentage_errors).item()
    return mean_absolute_percentage_error

In [17]:
import torch

def msle(y_true, y_pred):
    return torch.mean(torch.square(torch.log1p(y_true) - torch.log1p(y_pred)))

In [18]:
class Autoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 258),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(258, latent_dim),
           
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 258),
            nn.ReLU(),
            nn.Linear(258, input_dim)
           
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

class MODEL(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MODEL, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(32, output_dim)
        )

    def forward(self, x):
        return self.fc(x)

In [19]:
import torch
from torch.utils.data import DataLoader, TensorDataset


train_tensor = torch.tensor(train_new.drop('target',axis=1).to_numpy(), dtype=torch.float32)
Y_train_tensor = torch.tensor(train_new.target.to_numpy(), dtype=torch.float32)
test_tensor=torch.tensor(test_new.to_numpy(), dtype=torch.float32)
train_dataset = TensorDataset(train_tensor, Y_train_tensor)

train_loader = DataLoader(train_dataset, batch_size=300, shuffle=True)


In [20]:
autoencoder = Autoencoder(input_dim=train_new.shape[1]-1, latent_dim=200)

autoencoder_optimizer = optim.Adam(autoencoder.parameters(), lr=0.01)
autoencoder_criterion = nn.MSELoss()

for epoch in range(20):
    autoencoder.train()
    running_loss = 0.0
    for inputs, _ in tqdm(train_loader):
        autoencoder_optimizer.zero_grad()
        outputs = autoencoder(inputs)
        loss = autoencoder_criterion(outputs, inputs) 
        loss.backward()
        autoencoder_optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Autoencoder Loss: {running_loss / len(train_loader)}")

encoded_features = []
autoencoder.eval()
with torch.no_grad():
    for inputs, _ in train_loader:
        encoded = autoencoder.encoder(inputs)
        encoded_features.append(encoded)
encoded_features = torch.cat(encoded_features, dim=0)
encoded_test_features = []
autoencoder.eval()


  0%|          | 0/52 [00:00<?, ?it/s]

Epoch 1, Autoencoder Loss: 0.03973849590581197


  0%|          | 0/52 [00:00<?, ?it/s]

Epoch 2, Autoencoder Loss: 0.028119857852848675


  0%|          | 0/52 [00:00<?, ?it/s]

...

  0%|          | 0/52 [00:00<?, ?it/s]

Epoch 19, Autoencoder Loss: 0.02605792708121813


  0%|          | 0/52 [00:00<?, ?it/s]

Epoch 20, Autoencoder Loss: 0.026176248784535207


Autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=291, out_features=258, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=258, out_features=200, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=200, out_features=258, bias=True)
    (1): ReLU()
    (2): Linear(in_features=258, out_features=291, bias=True)
  )
)

In [21]:
encoded_test_features=autoencoder.encoder(test_tensor)

In [22]:
class  conv_net(nn.Module):

    def __init__(self, num_of_class):
        super(conv_net, self).__init__()

        self.model = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=3, stride=1, padding=1),#bathc*16*200
            nn.MaxPool1d(2), #batch*16*100

            nn.Conv1d(16, 64, kernel_size=3, stride=1, padding=1),#batch*64*100
            nn.MaxPool1d(2),#b*64*50

            nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.MaxPool1d(2),#b*128*25

        )

        self.linear = nn.Sequential(
            nn.Linear(3200,2000),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(2000,1500),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(1500, 1000),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(1000,num_of_class)
        )


    def forward(self,x):
        x = x.unsqueeze(1)
        x = self.model(x)
        # print(x.shape)
        x = x.view(x.size(0), -1)
        #x [b, 2944]
        # print(x.shape)
        x = self.linear(x)

        return x

In [23]:
class MODEL(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MODEL, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim,150),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(150, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64,output_dim)
            )

    def forward(self, x):
        return self.fc(x)

In [24]:
X_train_encoded, X_test_encoded, y_train, y_test = train_test_split(encoded_features, Y_train_tensor, test_size=0.2, random_state=42)

X_train_encoded_tensor = torch.tensor(X_train_encoded, dtype=torch.float32)
X_test_encoded_tensor = torch.tensor(X_test_encoded, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

train_dataset = TensorDataset(X_train_encoded_tensor, y_train_tensor)
train_loader2 = DataLoader(train_dataset, batch_size=500, shuffle=True)

test_dataset = TensorDataset(X_test_encoded_tensor, y_test_tensor)
test_loader2 = DataLoader(test_dataset, batch_size=300, shuffle=False)

C:\Users\Stepan\AppData\Local\Temp\ipykernel_11964\3242935974.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_encoded_tensor = torch.tensor(X_train_encoded, dtype=torch.float32)
C:\Users\Stepan\AppData\Local\Temp\ipykernel_11964\3242935974.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_encoded_tensor = torch.tensor(X_test_encoded, dtype=torch.float32)
C:\Users\Stepan\AppData\Local\Temp\ipykernel_11964\3242935974.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_tensor = torch.tensor(y_train, dtype=torch.f

In [25]:
train_dataset2 = TensorDataset(encoded_features, Y_train_tensor)
train_loader3 = DataLoader(train_dataset2, batch_size=500, shuffle=True)
test_3=encoded_test_features.detach()

In [87]:
#вручную подпбирал optimizer, learning_rate и функции активации и выбрал лучшее. также пробовал различные batch_size и количество эпох

In [26]:
predictor = conv_net(1)
optimizer = torch.optim.Adam(predictor.parameters(), lr=0.01)

for epoch in range(20):
    for x_train, y_train in tqdm(train_loader2):  # берем батч из трейн лоадера
        y_pred = predictor(x_train)  # делаем предсказания
        loss = rmse_loss(y_pred, y_train)  # считаем лосс
        loss.backward()  # считаем градиенты обратным проходом
        optimizer.step()  # обновляем параметры сети
        optimizer.zero_grad()  # обнуляем посчитанные градиенты параметров

    if epoch % 2 == 0:
        val_loss = []  # сюда будем складывать **средний по бачу** лосс
        val_mape = []
        with torch.no_grad():  # на валидации запрещаем фреймворку считать градиенты по параметрам
            for x_val, y_val in tqdm(test_loader2):  # берем батч из валидационного лоадера  # Пропускаем данные через автокодировщик
                y_pred = predictor(x_val) # делаем предсказания
                loss = rmse_loss(y_pred, y_val)  # считаем лосс
                val_loss.append(loss.item())  # добавляем в массив

                # Вычисляем MAPE
                MSLE = msle(y_pred,y_val)
                val_mape.append(MSLE)


        # печатаем метрики
        print(f"Epoch: {epoch}, loss: {np.mean(val_loss)}, MSLE: {np.mean(val_mape)}")

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

Epoch: 0, loss: 6136517.590909091, MSLE: 1.062049150466919


  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

...

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

Epoch: 18, loss: 4341183.045454546, MSLE: 0.3682723343372345


  0%|          | 0/25 [00:00<?, ?it/s]

In [27]:
#ЛУЧШАЯ МОДЕЛЬ

In [28]:
predictor = MODEL(200,1)
optimizer = torch.optim.Adam(predictor.parameters(), lr=0.01)

for epoch in range(50):
    for x_train, y_train in tqdm(train_loader2):  
        y_pred = predictor(x_train)  
        loss = rmse_loss(y_pred, y_train) 
        loss.backward() 
        optimizer.step()
        optimizer.zero_grad()  

    if epoch % 2 == 0:
        val_loss = []  
        val_msle = []
        with torch.no_grad():  
            for x_val, y_val in tqdm(test_loader2): 
                y_pred = predictor(x_val) 
                loss = rmse_loss(y_pred, y_val)  
                val_loss.append(loss.item())  

              
                MSLE = msle(y_pred,y_val)
                val_msle.append(MSLE)


       
        print(f"Epoch: {epoch}, loss: {np.mean(val_loss)}, MSLE: {np.mean(val_msle)}")

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

Epoch: 0, loss: 8142052.863636363, MSLE: 59.247989654541016


  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

...

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

Epoch: 48, loss: 4393239.636363637, MSLE: 0.3706918954849243


  0%|          | 0/25 [00:00<?, ?it/s]

In [29]:
predictor = MODEL(200,1)
optimizer = torch.optim.Adam(predictor.parameters(), lr=0.01)

for epoch in range(50):
    for x_train, y_train in tqdm(train_loader3):  
        y_pred = predictor(x_train)  
        loss = rmse_loss(y_pred, y_train) 
        loss.backward() 
        optimizer.step()
        optimizer.zero_grad()


  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

...

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

In [30]:
y_pred=predictor(test_3)

In [31]:
index_values = range(19999, 24377)
predicted_series = pd.Series([val[0] for val in y_pred.tolist()], index=index_values, name='id,prediction')
predicted_series.to_csv('sub.csv', header=True)